### Recurrent Neural Networks

# #PART 1 Data Preprocessing

In [0]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
#importing the dataset
dataset_train=pd.read_csv('/content/Google_Stock_Price_Train.csv')
# Selecting the first row and converting dataframe to numpy array as neural networks in keras take numpy array as input
training_set=dataset_train.iloc[:,1:2].values

In [0]:
#Feature Scaling
#Here we are doing Normalizing Scaling rather than standard Scaling
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
training_set_scaled=sc.fit_transform(training_set)

In [0]:
#Creating a data structure with 60 timesteps and 1 output
X_train=[]
y_train=[]
for i in range(60,1258):
  X_train.append(training_set_scaled[i-60:i,0])
  y_train.append(training_set_scaled[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)


In [0]:
#Reshaping 
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [0]:
#PART 2 BUILDING THE RNN
#importing Keras Libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout



In [0]:
# initializing the RNN
regressor=Sequential()

In [0]:
#Adding the first LSTM layer and some dropout Regularisation to avoid overfitting
#units signify no of neurons 
#return_sequence signify whether more layers will be not added  or not otherwise False
#input_shape signify only last two dimension of X_train
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))

In [0]:
# Adding the second LSTM layer and some dropout regularization
# No need to specify input_shape
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [0]:
# Adding the third LSTM layer
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [0]:
# Adding the fourth LSTM layer 
# No further layer is to add so return_sequence = False which is default 
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

In [0]:
#Adding the output layer
regressor.add(Dense(units=1))

In [0]:
# Compiling the RNN
# Since this is continuous prediction so loss=mean_squared_error
regressor.compile(optimizer='adam',loss='mean_squared_error')

In [0]:
#Fitting the RNN to Training Set
regressor.fit(X_train,y_train,epochs=100,batch_size=32)

In [0]:
#PART 3 Making the prediction and visualizing the result
#Getting the real stock price of 2017 i.e y_test
dataset_test=pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price=dataset_test.iloc[:,1:2].values                                 



In [0]:
#Getting the predicted stock price of 2017 i.e x_test
dataset_total=pd.concat((dataset_train['Open'],dataset_test['Open']),axis=0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


In [0]:
predicted_stock_price

In [0]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()